1. Installing required packages

In [ ]:
%pip install --upgrade --quiet keras tensorflow pandas numpy matplotlib

import keras
import tensorflow
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

print(f'keras version: {keras.__version__}')
print(f'tensorflow version: {tensorflow.__version__}')
print(f'pandas version: {pd.__version__}')
print(f'numpy version: {np.__version__}')
print(f'matplotlib version: {matplotlib.__version__}')

2. Load test data

In [ ]:
import pandas as pd

house_data = pd.read_json('data/house_data.json')
weather_data_annual = pd.read_json('data/weather_data_annual.json')
annual_true_output = pd.read_json('data/true_output_annual.json')

3. Convert Pandas Dataframe to Numpy matrix

In [ ]:
import json

def __convert_to_input_house(house_df):
    data_dict = pd.read_csv('data/data_dictionary.csv')
    non_zero_std_column_list = json.loads(open('data/non_zero_std_column_list.json').read())
    columns = house_df.columns

    new_house_data = house_df.copy()
    new_house_dict_list = new_house_data.to_dict(orient='records')

    converted_house_dict_list = []
    for house_dict in new_house_dict_list:
        converted_house_dict = {}
        for key, value in house_dict.items():
            data_dict_rows = data_dict[data_dict['hpxml_key'] == key]
            if not data_dict_rows.empty:
                data_dict_row = data_dict_rows.iloc[0]
                if data_dict_row['value'] in ['__double__', '__int__']:
                    converted_house_dict[data_dict_row['name']] = float(value)
                else:
                    for idx, data_dict_row in data_dict_rows.iterrows():
                        if data_dict_row['value'] == value:
                            converted_house_dict[data_dict_row['name']] = 1

        converted_house_dict_list.append(converted_house_dict)

    converted_house_df = pd.DataFrame(converted_house_dict_list)

    # if any column in non_zero_std_column_list does not exist in the converted_house_df, add it with value 0
    for column in non_zero_std_column_list:
        if column not in converted_house_df.columns:
            converted_house_df[column] = 0

    # fill NaN values with 0
    converted_house_df.fillna(0, inplace=True)

    # pick only the columns that are in the non_zero_std_column_list
    converted_house_df = converted_house_df[non_zero_std_column_list]

    return converted_house_df

def __convert_to_input_weather_annual(weather_annual_df):
    return weather_annual_df

def __convert_to_input_true_output_annual(true_output_annual_df):
    return true_output_annual_df

def convert_to_input(df, mode):
    assert mode in ['house', 'weather_annual', 'true_output_annual'], 'Invalid mode'
    if mode == 'house':
        return __convert_to_input_house(df)
    elif mode == 'weather_annual':
        return __convert_to_input_weather_annual(df)
    elif mode == 'true_output_annual':
        return __convert_to_input_true_output_annual(df)
    else:
        raise ValueError('Invalid mode')
    
house_data_input = convert_to_input(house_data, "house")
weather_data_annual_input = convert_to_input(weather_data_annual, "weather_annual")
annual_true_output_input = convert_to_input(annual_true_output, "true_output_annual")

4. Load the trained model

In [ ]:
annual_model = keras.models.load_model('models/annual.keras')
annual_model.summary()

5. Create the input data matrix

In [ ]:
# concatenate the house and weather data side by side
annual_input_data = np.concatenate((house_data_input, weather_data_annual_input), axis=1)

6. Run the model

In [ ]:
annual_output_data = annual_model.predict(annual_input_data, batch_size=annual_input_data.shape[0])

7. Compare the results

In [ ]:
# draw x-y plot of the predicted and true output
plt.scatter(annual_output_data, annual_true_output_input)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Annual Energy Consumption Prediction (Annual MMBTU)')

# draw a line from the bottom left to the top right
maxval = max(plt.xlim()[1], plt.ylim()[1])
plt.plot([0, maxval], [0, maxval], color='red')

plt.show()